In [1]:
from dotenv import load_dotenv

assert load_dotenv("../../.env")

In [2]:
import ragna

ragna.__version__

'0.1.dev29+g03721c7.d20231009084220'

In [3]:
from ragna import demo_config, Config

demo_config

Config(local_cache_root=PosixPath('/home/philip/.cache/ragna'), state_database_url='sqlite://', queue_database_url='memory', ragna_api_url='http://127.0.0.1:31476', ragna_ui_url='http://127.0.0.1:31477', document_class=<class 'ragna.core.LocalDocument'>, upload_token_secret='16d97e7eb82eaa866805280bf3e4302b8d459fa70e1c77753fdaa8ad047fa55f', upload_token_ttl=30, registered_source_storage_classes={'Ragna/DemoSourceStorage': <class 'ragna.source_storage._demo.RagnaDemoSourceStorage'>}, registered_assistant_classes={'Ragna/DemoAssistant': <class 'ragna.assistant._demo.RagnaDemoAssistant'>})

In [4]:
document_path = "demo_document.txt"

with open(document_path, "w") as file:
    file.write("Ragna is an open-source RAG orchestration app\n")

In [5]:
from ragna.core import Rag
from ragna.assistant import (
    RagnaDemoAssistant,
    OpenaiGpt35Turbo16kAssistant,
    OpenaiGpt4Assistant,
)
from ragna.source_storage import (
    ChromaSourceStorage,
    RagnaDemoSourceStorage,
    LanceDBSourceStorage,
)

rag = Rag(demo_config)

async with rag.chat(
    documents=[document_path],
    source_storage=RagnaDemoSourceStorage,
    assistant=RagnaDemoAssistant,
) as chat:
    prompt = "What is Ragna?"
    answer = await chat.answer(prompt)

print(answer)

I can't really help you with your prompt:

> What is Ragna?

I can at least show you the sources that I was given:

- demo_document.txt: Ragna is an open-source RAG orchestration app


In [6]:
import itertools
import asyncio
from pprint import pprint

source_storages = [ChromaSourceStorage, LanceDBSourceStorage]
assistants = [OpenaiGpt35Turbo16kAssistant, OpenaiGpt4Assistant]


async def answer_prompt(source_storage, assistant):
    async with rag.chat(
        documents=[document_path],
        source_storage=source_storage,
        assistant=assistant,
    ) as chat:
        return await chat.answer(prompt)


experiments = {
    (source_storage.display_name(), assistant.display_name()): answer_prompt(
        source_storage, assistant
    )
    for source_storage, assistant in itertools.product(source_storages, assistants)
}

pprint(experiments)

{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7ff6b6164ac0>,
 ('Chroma', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7ff6b6164b40>,
 ('LanceDB', 'OpenAI/gpt-3.5-turbo-16k'): <coroutine object answer_prompt at 0x7ff6b6164bc0>,
 ('LanceDB', 'OpenAI/gpt-4'): <coroutine object answer_prompt at 0x7ff6b6164c40>}


In [7]:
results = dict(zip(experiments.keys(), await asyncio.gather(*experiments.values())))
pprint(results)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{('Chroma', 'OpenAI/gpt-3.5-turbo-16k'): Message(content='Ragna is an open-source RAG (Response Analysis Graph) orchestration app. It is designed to help users create conversational AI applications by providing a framework for managing and orchestrating the flow of conversations. Ragna allows developers to define conversation flows, handle user inputs, and generate dynamic responses based on predefined rules and logic. It is built on top of the Rasa framework and provides additional features and functionalities to simplify the development process.', role=<MessageRole.ASSISTANT: 'assistant'>, sources=[Source(id='4afc4718-c8b0-4d3b-b103-24ecfce42a99', document=<ragna.co